# 🔬 ISIC Cilt Lezyonu Analizi - Final Versiyon

**Mustafa Engin Dalgıç | 254309502 | Üsküdar Üniversitesi**

---

## ▶️ ÇALIŞTIRMA:

1. **Sağ üstten "Bağlan" (Connect) tıklayın** ⚡
2. **Runtime > Run all** tıklayın
3. Bekleyin, grafikler otomatik görünecek! 📊

---

In [ ]:
print("🚀 ISIC Cilt Lezyonu Analizi Başlıyor...")
print("Lütfen bekleyin...\n")

In [ ]:
# ==================== KÜTÜPHANELER ====================
print("📦 Kütüphaneler yükleniyor...\n")

import sys
import os
import warnings
warnings.filterwarnings('ignore')

# OpenCV
try:
    import cv2
    print(f"✅ OpenCV {cv2.__version__}")
except:
    print("⚠️ OpenCV kuruluyor...")
    !pip install opencv-python -q
    import cv2
    print(f"✅ OpenCV {cv2.__version__} kuruldu")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
import zipfile
import shutil

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

print(f"✅ NumPy {np.__version__}")
print(f"✅ Pandas {pd.__version__}")
print(f"✅ Matplotlib hazır")
print("\n✅ Tüm kütüphaneler hazır!\n")

In [ ]:
# ==================== PROJE İNDİRME (ZIP) ====================
print("📥 GitHub'dan proje indiriliyor (ZIP)...\n")

# Eski klasörü temizle
!rm -rf UU_DIP_ISIC_SKIN_LESION_ANALYSIS 2>/dev/null
!rm -f project.zip 2>/dev/null

# ZIP olarak indir (Git klonlama sorunu yok)
zip_url = "https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS/archive/refs/heads/main.zip"

try:
    print("📦 ZIP dosyası indiriliyor...")
    urllib.request.urlretrieve(zip_url, "project.zip")
    
    print("📂 ZIP açılıyor...")
    with zipfile.ZipFile("project.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    
    # Klasör adını düzelt
    if os.path.exists("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main"):
        os.rename("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main", "UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
    
    print("✅ Proje indirildi!\n")
    
except Exception as e:
    print(f"⚠️ ZIP indirme hatası: {e}")
    print("Git klonlama deneniyor...")
    !git clone https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS.git -q
    print("✅ Git ile indirildi!\n")

# Proje klasörüne geç
%cd UU_DIP_ISIC_SKIN_LESION_ANALYSIS

In [ ]:
# ==================== VERİ SETİ KONTROLÜ ====================
print("🔍 Veri seti kontrol ediliyor...\n")

DATA_PATH = "ISIC"

# Görüntüleri bul
image_files = []
if os.path.exists(DATA_PATH):
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image_files.append(os.path.join(root, file))

if len(image_files) == 0:
    print("❌ HATA: ISIC klasöründe görüntü bulunamadı!")
    print("\n💡 GitHub reposuna ISIC klasörünü ekleyin.")
else:
    print(f"✅ {len(image_files)} görüntü bulundu!")
    print(f"\n📁 İlk 5 dosya:")
    for i, f in enumerate(image_files[:5], 1):
        print(f"   {i}. {os.path.basename(f)}")
    if len(image_files) > 5:
        print(f"   ... ve {len(image_files)-5} tane daha\n")

In [ ]:
# ==================== VERİ YÜKLEME ====================
print("📊 Veri seti yükleniyor...\n")

def load_images_safe(image_paths, max_samples=50):
    """Güvenli veri yükleme"""
    data = []
    
    sample_paths = image_paths[:max_samples] if max_samples else image_paths
    
    for i, img_path in enumerate(sample_paths):
        try:
            img = Image.open(img_path)
            width, height = img.size
            
            # Kanal sayısı
            if img.mode == 'RGB':
                channels = 3
            elif img.mode == 'L':
                channels = 1
            elif img.mode == 'RGBA':
                channels = 4
            else:
                channels = len(img.getbands())
            
            file_size = os.path.getsize(img_path) / 1024
            
            data.append({
                'filename': os.path.basename(img_path),
                'filepath': img_path,
                'width': width,
                'height': height,
                'channels': channels,
                'file_size_kb': round(file_size, 2)
            })
            
            if (i+1) % 25 == 0:
                print(f"   {i+1}/{len(sample_paths)} yüklendi...")
                
        except Exception as e:
            continue
    
    return pd.DataFrame(data)

# İlk 50 görüntü (hızlı test)
train_df = load_images_safe(image_files, max_samples=50)

if len(train_df) == 0:
    print("\n❌ Veri yüklenemedi!")
else:
    print(f"\n✅ {len(train_df)} görüntü yüklendi!")
    print("\n📋 İlk 3 satır:")
    display(train_df.head(3))

In [ ]:
# ==================== İSTATİSTİKLER ====================
print("\n📊 İstatistikler:\n")

print(f"📐 Çözünürlük:")
print(f"   Ortalama: {train_df['width'].mean():.0f}x{train_df['height'].mean():.0f}")
print(f"   Min: {train_df['width'].min()}x{train_df['height'].min()}")
print(f"   Max: {train_df['width'].max()}x{train_df['height'].max()}")

print(f"\n💾 Boyut:")
print(f"   Ortalama: {train_df['file_size_kb'].mean():.2f} KB")
print(f"   Toplam: {train_df['file_size_kb'].sum()/1024:.2f} MB")

In [ ]:
# ==================== GÖRÜNTÜ GÖSTERİMİ ====================
print("\n🖼️ Görüntüler gösteriliyor...\n")

def resize_img(img, max_size=400):
    h, w = img.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        return cv2.resize(img, None, fx=scale, fy=scale)
    return img

# İlk 3 görüntü
n = min(3, len(train_df))
samples = train_df.iloc[:n]

fig, axes = plt.subplots(n, 2, figsize=(10, n*2.5))
if n == 1:
    axes = axes.reshape(1, -1)

fig.suptitle('✅ RGB vs Grayscale', fontsize=14, fontweight='bold')

for idx, (i, row) in enumerate(samples.iterrows()):
    img = cv2.imread(row['filepath'])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = resize_img(img_rgb)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    axes[idx, 0].imshow(img_rgb)
    axes[idx, 0].set_title(f"RGB - {row['filename'][:20]}")
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(img_gray, cmap='gray')
    axes[idx, 1].set_title(f"Grayscale - {row['width']}x{row['height']}")
    axes[idx, 1].axis('off')

plt.tight_layout()
plt.show()

print("✅ Görüntüler gösterildi!")

In [ ]:
# ==================== HİSTOGRAM ====================
print("\n📊 Histogram analizi...\n")

# İlk görüntü için
first = train_df.iloc[0]
img = cv2.imread(first['filepath'])
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_rgb = resize_img(img_rgb)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

# Histogram hesapla
hist_r = cv2.calcHist([img_rgb], [0], None, [256], [0, 256])
hist_g = cv2.calcHist([img_rgb], [1], None, [256], [0, 256])
hist_b = cv2.calcHist([img_rgb], [2], None, [256], [0, 256])
hist_gray = cv2.calcHist([img_gray], [0], None, [256], [0, 256])

# Göster
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
fig.suptitle('Histogram Analizi', fontsize=14, fontweight='bold')

axes[0].plot(hist_r, 'r-', alpha=0.7, label='Red')
axes[0].plot(hist_g, 'g-', alpha=0.7, label='Green')
axes[0].plot(hist_b, 'b-', alpha=0.7, label='Blue')
axes[0].set_title('RGB Histogram')
axes[0].set_xlabel('Piksel Değeri')
axes[0].set_ylabel('Frekans')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(hist_gray, 'k-')
axes[1].set_title('Grayscale Histogram')
axes[1].set_xlabel('Piksel Değeri')
axes[1].set_ylabel('Frekans')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Histogram tamamlandı!")

In [ ]:
# ==================== CLAHE ====================
print("\n✨ CLAHE (Kontrast İyileştirme)...\n")

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img_clahe = clahe.apply(img_gray)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('CLAHE - Kontrast İyileştirme', fontsize=14, fontweight='bold')

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Orijinal')
axes[0].axis('off')

axes[1].imshow(img_clahe, cmap='gray')
axes[1].set_title('CLAHE')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("✅ CLAHE tamamlandı!")

---

## 🎉 TEST BAŞARILI!

### ✅ Tamamlanan:
1. Veri yükleme
2. Görüntü gösterimi
3. Histogram analizi
4. CLAHE uygulaması

### 🚀 TAM PROGRAM:

Aşağıdaki hücreyi çalıştırarak 45+ grafik elde edin:

In [ ]:
print("🚀 TAM PROGRAM BAŞLIYOR (10-15 dk)...\n")
!python isic_project.py

---

### 💬 İletişim

**Mustafa Engin Dalgıç**  
📧 engindalgic86@gmail.com  
🎓 Üsküdar Üniversitesi  
🔗 [GitHub](https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS)

⭐ **Yıldız vermeyi unutmayın!** ⭐